In [8]:
import yaml
import h5py
import numpy as np
import pickle as pkl

import matplotlib.pyplot as plt

from rdkit import Chem

# Molecular structure elucidation from NMR spectra: Part 1

In this exercise we will examine one of the applications of the Transformer architecture we covered in lecture today. The example here is based on recently published work by [Hu et. al.](https://pubs.acs.org/doi/10.1021/acscentsci.4c01132) where a machine learning (ML) model was developed to tackle the inverse problem of elucidating molecular structure from routintely collected 1D NMR spectra (see below). If you have taken an organic chemistry class, you likely had to assign peaks in $^1$H and $^{13}$C NMR spectra to functional groups (i.e., substructures) and your experimental colleagues routinely conduct NMR measurements to characterize their synthesized products. NMR spectra serve as useful molecular fingerprints, but, even with a set of tabulated shifts characteristic of different functional groups or databases to cross-search spectra, the task of determining molecular structure from NMR spectra can be difficult for larger molecules.

<div style="text-align: center;">
    <img src="figures/toc.jpeg" width="85%"/>
    <figcaption><i>Figure 1: (Top) Overview of our ML model that takes NMR spectra as input to predict for the corresponding molecular structure. (Bottom) Downstream parts of the model are pretrained to predict the molecular structure based on a list of substructures that make up the molecule.</i></figcaption>
</div>

We will attempt to better inform this structure elucidation task with an ML model. To accomplish this, we will represent our molecular structures with [SMILES](https://chem.libretexts.org/Courses/Fordham_University/Chem1102%3A_Drug_Discovery_-_From_the_Laboratory_to_the_Clinic/05%3A_Organic_Molecules/5.08%3A_Line_Notation_(SMILES_and_InChI)) strings, a character-based representation of a molecular graph, and develop a ML model to predict for each character one-by-one autoregressively. In this way, we are effectively "translating" from our NMR spectra to SMILES and will leverage a Transformer in order to do so.

Since NMR spectra are relatively expensive to generate/source, we will first develop a transformer-based model that just predicts molecular structure given what substructures are present (i.e., the bottom half of Figure 1). In the next half of this exercise, we will use this model to initialize part of a larger model that goes from spectral data inputs to substructure prediction embeddings that are then passed through the transformer we construct here to ultimately obtain molecular structure predictions. Data to train this substructure-to-structure model is a lot easier to source and label, since there exist large datasets of chemically feasible molecular structures and identifying what substructures a given molecule contains is relatively straightforward. The training of the overall spectra to structure model that we will conduct later will be considerably easier if we can pretrain this part of the model.

<div style="text-align: center;">
    <img src="figures/architecture1.png" width="35%"/>
    <figcaption><i>Figure 2: Transformer architecture used for our substructure to structure model</i></figcaption>
</div>

Figure 2 above summarizes the architecture of the transformer we will construct in the first half of this exercise. In short, it will take as input both an array which will indicate what substructures are present in the molecule and a prefix of SMILES characters to make the prediction for the next character in the predicted SMILES string for the molecule. By repeatedly sampling the model with a growing prefix of SMILES character inputs we, in an autoregressive fashion, can predict for a complete SMILES string for the molecule.

## Local installation and setup

For this exercise we will train our models using GPU resources on the NYU Shanghai cluster using the NMR2Struct software package while deconstructing the models and performing analyses locally on your personal laptops. This is because training is computationally quite demanding but evaluating a trained model and the analyses we will conduct is not.

To install NMR2Struct on your personal laptop please visit the [GitHub repo](https://github.com/MarklandGroup/NMR2Struct/tree/main) and follow the install instructions. Since your laptop might not have a dedicated GPU, please modify the environment.yaml file to install a CPU only version of pytorch (i.e., change "pytorch::pytorch-cuda=12.1" to "pytorch").

The dataset we will use to train and evaluate our model has been taken from this [Zenodo archive](https://zenodo.org/records/13892026). The training of the model you will subsequently perform on the NYU Shanghai cluster will make use of this full dataset. On your personal laptop, you need to only download the "data" subfolder provided in the Google Drive for this exercise to the same directory with this Jupyter notebook.

## Training our substructure to structure model

Since the training of our substructure to structure model will take some time, let us first get that started before we dive into the other details. Training our model will be straightforward since we will use the full implementation of the model provided by the NMR2Struct software package. To start training the model:

1. Login to the NYU Shanghai cluster and navigate to where you would like to perform the training of the model.

```
ssh <user>@hpclogin.shanghai.nyu.edu
```

2. Copy over the pre-prepared training run directory:

```
rsync -avz --exclude="datasets" --exclude="NMR2Struct" /gpfsnyu/home/mc10050/nmr2mol .
```

3. (Optional) The settings for specifying the model architecture and training hyperparameters. Feel free to try modifying things (e.g., like the embedding dimension of the model d_model) if you would like to try something aside from the default settings! If you have any questions about what the settings refer to, do not hesitate to ask.
  
4. Submit the training run to the job scheduler:

```
cd nmr2mol/exercise_part1/
sbatch -J sub2struct-train submit.sh
```

5. You can monitor the progress training your model by downloading the tfevents file generated to your local machine and using Tensorboard as show below and navigating in your web browser to the indicated URL (e.g., "http://localhost:6006/"). Note that the losses used to optimize our model parameters and that are visualized via Tensorboard are the standard cross-entropy losses

```
rsync -avz "<user>@hpclogin.shanghai.nyu.edu:~/nmr2mol/exercise_part1/checkpoints/events.out.*" model1/
conda activate NMR_env
tensorboard --logdir=model1/

```

Now you just need to wait for your job to run and your model will then start training, which will take on the order of an hour. In the meantime, let us understand the details of the model we are training by building our own version of it from scratch using Pytorch.


In [ ]:
%%bash
rsync -avz --exclude="datasets" --exclude="NMR2Struct" /gpfsnyu/home/mc10050/nmr2mol /gpfsnyu/scratch/ys6132/2025_summer_school_ML4MS/day3

sending incremental file list
drwxrwxrwx          4,096 2025/07/16 12:41:58 nmr2mol
drwxrwxrwx          4,096 2025/07/16 12:49:43 nmr2mol/exercise_part1
-rwxrwxrwx            517 2025/07/16 12:37:54 nmr2mol/exercise_part1/conda-env.sh
-rwxrwxrwx          1,295 2025/07/13 14:17:05 nmr2mol/exercise_part1/full_train_config.yaml
-rwxrwxrwx            183 2025/07/16 12:39:10 nmr2mol/exercise_part1/submit.sh
drwxrwxrwx          4,096 2025/07/16 12:34:56 nmr2mol/exercise_part1/data
lrwxrwxrwx             51 2025/07/16 12:34:56 nmr2mol/exercise_part1/data/alphabet.npy -> /gpfsnyu/home/mc10050/nmr2mol/datasets/alphabet.npy
lrwxrwxrwx             49 2025/07/16 12:34:56 nmr2mol/exercise_part1/data/smiles.npy -> /gpfsnyu/home/mc10050/nmr2mol/datasets/smiles.npy
lrwxrwxrwx             54 2025/07/16 12:34:56 nmr2mol/exercise_part1/data/split_indices.p -> /gpfsnyu/home/mc10050/nmr2mol/datasets/split_indices.p
lrwxrwxrwx             55 2025/07/16 12:34:56 nmr2mol/exercise_part1/data/substructures.h5 -

## Checking the dataset

The dataset we will be working with consists of a total of 146595 molecules with each molecule containing up to 19 heavy atoms. The following files contain the different components of our dataset:

1. <i>smiles.npy</i> contains all of the SMILES strings for each molecule.
2. <i>substructures_957.p</i> contains all the SMARTS strings defining each of the 957 substructures that we are using for our substructure to structure prediction model. SMARTS are similar to SMILES but are more flexible (e.g., can define coordination of atoms) and hence more well suited for defining our substructures.
3. <i>substructures.h5</i> contains the substructure labeling for each molecule.

Below we load the SMILES string and substructure labels for one molecule from our dataset

In [10]:
i = 10

ismi = np.load('data/smiles_test.npy')[i].decode('UTF-8')

substruct_list = pkl.load((open('data/substructures_957.p', 'rb')))
hf = h5py.File('data/substructures_test.h5', 'r')
isubs = hf['substructure_labels'][i]
hf.close()

FileNotFoundError: [Errno 2] No such file or directory: 'data/smiles_test.npy'

The following block uses the rdKit software package to visualize the molecule as specified by its SMILES string

In [ ]:
print(ismi)
imol = Chem.MolFromSmiles(ismi)
Chem.Draw.MolToImage(imol)

The following block uses the rdKit software package to visualize which of the 957 substructures the molecule above has, with the corresponding SMARTS specifying the substructure printed as well. The dashed lines indicate that the bond type (e.g., single, double, or triple) is unspecified for that pair. The model that we will subsequently construct will take the set of substructures below as input to attempt at predicting the full molecular structure above 

In [ ]:
isubstruct_list = [Chem.MolFromSmarts(x) for x in substruct_list[isubs]]
Chem.Draw.MolsToGridImage(isubstruct_list,molsPerRow=8,subImgSize=(150,150),legends=[str(x) for x in substruct_list[isubs]])    

## Building our substructure to structure model

In [ ]:
import math
import random
import torch
from typing import Tuple, Callable, Optional, Any
from torch import nn, Tensor

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

Our model first processes the input data, both the arrays of substructures and the SMILES prefix, by passing them through embedding layers (Figure 2). The purpose of the embedding layers is to encode the tokens of the input data (e.g., words in a sentence or SMILES characters) into a numerical vector representation more amenable for fitting our models to (e.g., word2vec). The embedding layers themselves are learnable.

As part of the embedding layer, we will also apply positional encoding. In doing so, we effectively apply a position dependent function to the input arrays. This has the purpose of injecting some information about the ordering of tokens in our input data. Our model will make use of sinusoidal positional encodings of the following form

$$PE_{pos,i} = sin(pos/10000^{2i/d_{model}})$$
$$PE_{pos,2i+1} = cos(pos/10000^{2i/d_{model}})$$

In the code block below, complete the implementation of the PositionaEncoding class. You can verify whether you have correctly implemented things by checking if the example gives you something like the following contour plot (for an input with dimension 957 and an output with dimension $d_{model}=128$):

<div style="text-align: center;">
    <img src="figures/positionalencoding.png" width="30%"/>
</div>

In [ ]:
class PositionalEncoding(nn.Module):

    """ Positional encoding with option of selecting specific indices to add """

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 30000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        ### MODIFY BELOW ###

        ### MODIFY ABOVE ###
    
    def forward(self, x: Tensor, ind: Tensor) -> Tensor:
        '''
        Args:
            x: Tensor, shape [batch_size, seq_len, embedding_dim]
            ind: Tensor, shape [batch_size, seq_len] or NoneType
        '''
        if ind is not None:
            added_pe = self.pe[torch.arange(1).reshape(-1, 1), ind, :]
            x = x + added_pe
        else:
            ### MODIFY BELOW ###

            ### MODIFY ABOVE ###
        return self.dropout(x)


posencode = PositionalEncoding(128, dropout=0)
x = torch.ones((1,957,128))
y = posencode(x, None)
plt.contourf(y.detach().numpy()[0].T)
plt.show()

With the positional encoder implemented, let us now combine it with embedding layers to complete the first input processing steps for our model. Implement the functions below, which will specify the embedding and encoding of our source (i.e., substructure inputs) and targets (i.e., SMILES string prefix). The two embedding/encoding forward procedures should effectively be the same but we will separate them to keep things fully general.

I would suggest just using the [nn.Embedding](https://docs.pytorch.org/docs/stable/generated/torch.nn.Embedding.html) Pytorch module here.

In [ ]:
def src_fwd_fxn_basic(src: Tensor,
                      d_model: int,
                      src_embed: nn.Module = nn.Embedding,
                      src_pad_token: int = 0,
                      pos_encoder: nn.Module = nn.Embedding) -> Tuple[Tensor, Optional[Tensor]]:
    """Forward processing for source tensor in Transformer for substructure to structure problem
    Args:
        src: The unembedded source tensor, raw input into the forward() method, 
            shape (batch_size, seq_len)
        d_model: The dimensionality of the model
        src_embed: The source embedding layer
        src_pad_token: The source padding token index
        pos_encoder: The positional encoder layer
    """
    if not isinstance(src_embed, nn.Embedding):
        src_key_pad_mask = None
    elif src_pad_token is not None:
        src_key_pad_mask = (src == src_pad_token).bool().to(src.device)

    ### MODIFY BELOW ###

    
    ### MODIFY ABOVE ###
    return src, src_key_pad_mask


def tgt_fwd_fxn_basic(tgt: Tensor,
                      d_model: int, 
                      tgt_embed: nn.Module = nn.Embedding,
                      tgt_pad_token: int = 21,
                      pos_encoder: nn.Module = nn.Embedding) -> Tuple[Tensor, Optional[Tensor]]:
    """Standard forward processing function for target tensor used in the Transformer network 
    Args:
        tgt: The unembedded target tensor, raw input into the forward() method,
            shape (batch_size, seq_len)
        d_model: The dimensionality of the model
        tgt_embed: The target embedding layer
        tgt_pad_token: The target padding token index
        pos_encoder: The positional encoder layer
    """
    if tgt_pad_token is not None:
        tgt_key_pad_mask = (tgt == tgt_pad_token).bool().to(tgt.device)
    else:
        tgt_key_pad_mask = None

    ### MODIFY BELOW ###

    ### MODIFY ABOVE ###
    return tgt, tgt_key_pad_mask

Now let us construct our Transformer model. We will be employing a layer of abstraction to match the design of the model implementation in NMR2Struct, but you will only be asked here to complete the implementation of the Transformer class (not the abstract TransformerModel class). In particular, to complete the Transformer class you will need to initialize it properly and also implement the forward routine. Here I would suggest you make use of the [nn.Transformer](https://docs.pytorch.org/docs/stable/generated/torch.nn.Transformer.html) from Pytorch as well as the embedding routines you already completed.

In [ ]:
class Transformer(nn.Module):

    model_id = "Transformer"
    
    def __init__(self, src_embed: nn.Module, tgt_embed: nn.Module,
                 src_pad_token: int, tgt_pad_token: int, 
                 src_forward_function: Callable[[Tensor, nn.Module, int, Optional[nn.Module]], Tuple[Tensor, Optional[Tensor]]],
                 tgt_forward_function: Callable[[Tensor, nn.Module, int, Optional[nn.Module]], Tuple[Tensor, Optional[Tensor]]],
                 d_model: int = 512, nhead: int = 8, num_encoder_layers: int = 6, num_decoder_layers: int = 6,
                 dim_feedforward: int = 2048, dropout: float = 0.1, activation: str = 'relu', custom_encoder: Optional[Any] = None,
                 custom_decoder: Optional[Any] = None, target_size: int = 50, source_size: int = 957,
                 layer_norm_eps: float = 1e-05, batch_first: bool = True, norm_first: bool = False, 
                 device: torch.device = None, dtype: torch.dtype = torch.float):
        
        r"""Most parameters are standard for the PyTorch transformer class. A few specific ones that have been added:

        src_embed: The embedding module for the src tensor passed to the model
        tgt_embed: The embedding module for the tgt tensor passed to the model
        src_pad_token: The index used to indicate padding in the source sequence
        tgt_pad_token: The index used to indicate padding in the target sequence
        src_forward_function: A function that processes the src tensor using the src embedding, src pad token, and positional encoding to generate
            the embedded src and the src_key_pad_mask
        tgt_forward_function: A function that processes the tgt tensor using the tgt embedding, tgt pad token, and positional encoding to generate
            the embedded tgt and the tgt_key_pad_mask
        target_size (int): Size of the target alphabet (including start, stop, and pad tokens).
        source_size (int): Size of the source alphabet (including start, stop, and pad tokens).
        batch_first is set to be default True, more intuitive to reason about dimensionality if batching 
            dimension is first
        """
        super().__init__()

        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_fwd_fn = src_forward_function
        self.tgt_fwd_fn = tgt_forward_function
        self.src_pad_token = src_pad_token
        self.tgt_pad_token = tgt_pad_token

        self.tgt_size = target_size
        self.src_size = source_size
        self.d_model = d_model
        self.dtype = dtype
        self.device = device

        ### MODIFY BELOW ###

        ### MODIFY ABOVE ###

    def _get_tgt_mask(self, size):
        #Generate a mask for the target to preserve autoregressive property. Note that the mask is 
        #   Additive for the PyTorch transformer
        mask = torch.tril(torch.ones(size, size, dtype = self.dtype, device = self.device))
        mask = mask.masked_fill(mask == 0, float('-inf'))
        mask = mask.masked_fill(mask == 1, float(0.0))
        return mask
    
    def _sanitize_forward_args(self, 
                               x: Tuple[Tensor, Tuple],
                               y: Tuple[Tensor, Tensor]) -> Tuple[Tensor, Tensor, Tensor]:
        '''Processes raw x, y inputs to the transformer for use in forward()
        Args:
            x: Tuple of a tensor (input) and the set of smiles strings (smiles)
            y: A tuple of a the shifted target tensor and full target tensor
        '''
        #Unpack the tuples
        inp, _ = x
        shifted_y, _ = y
        if isinstance(self.src_embed, nn.Embedding):
            inp = inp.long()
        if isinstance(self.tgt_embed, nn.Embedding):
            shifted_y = shifted_y.long()
        return inp, shifted_y
    
    #Sketch of what the forward function could look like with more abstraction
    def forward(self, 
                x: Tuple[Tensor, Tuple], 
                y: Tuple[Tensor, Tensor]) -> Tensor:
        src, tgt = self._sanitize_forward_args(x, y)
        tgt_mask = self._get_tgt_mask(tgt.size(1)).to(tgt.device)
        ### MODIFY BELOW ###

        ### MODIFY ABOVE ###
        return out

    def get_loss(self,
                 x: Tuple[Tensor, Tuple], 
                 y: Tuple[Tensor], 
                 loss_fn: Callable[[Tensor, Tensor], Tensor]) -> Tensor:
        """
        Unpacks the input and obtains the loss value
        Args:
            x: Tuple of a tensor (input) and the set of smiles strings (smiles)
            y: A tuple of a the shifted target tensor and full target tensor
            loss_fn: The loss function to use for the model, with the signature
                tensor, tensor -> tensor
        """
        pred = self.forward(x, y)
        pred = pred.permute(0, 2, 1)
        _, full_y = y
        if isinstance(self.tgt_embed, nn.Embedding):
            full_y = full_y.long()
        loss = loss_fn(pred, full_y.to(self.device))
        return loss

class TransformerModel(nn.Module):
    """ Example model wrapper for transformer network """

    def __init__(self, 
                 src_embed: str, 
                 src_embed_options: dict,
                 tgt_embed: str, 
                 tgt_embed_options: dict,
                 src_pad_token: int, 
                 tgt_pad_token: int,
                 src_forward_function: str, 
                 tgt_forward_function: str, 
                 freeze_components: Optional[list] = None,
                 d_model: int = 512, 
                 nhead: int = 8, 
                 num_encoder_layers: int = 6, 
                 num_decoder_layers: int = 6,
                 dim_feedforward: int = 2048, 
                 dropout: float = 0.1, 
                 activation: str = 'relu', 
                 custom_encoder: Optional[Any] = None,
                 custom_decoder: Optional[Any] = None, 
                 target_size: int = 50, 
                 source_size: int = 957,
                 layer_norm_eps: float = 1e-05, 
                 batch_first: bool = True, 
                 norm_first: bool = False, 
                 device: torch.device = None, 
                 dtype: torch.dtype = torch.float):
        r"""Most parameters are standard for the PyTorch transformer class. A few specific ones that have been added:

        src_embed: The name of the embedding module for the src tensor passed to the model
        src_embed_options: Dictionary of options for the src embedding module
        tgt_embed: The name of the embedding module for the tgt tensor passed to the model
        tgt_embed_options: Dictionary of options for the tgt embedding module
        src_pad_token: The index used to indicate padding in the source sequence
        tgt_pad_token: The index used to indicate padding in the target sequence
        src_forward_function: Name of the function that processes the src tensor using the src embedding, src pad token, and positional encoding to generate
            the embedded src and the src_key_pad_mask
        tgt_forward_function: Name of the function that processes the tgt tensor using the tgt embedding, tgt pad token, and positional encoding to generate
            the embedded tgt and the tgt_key_pad_mask
        freeze_components: List of component names to freeze
        target_size (int): Size of the target alphabet (including start, stop, and pad tokens).
        source_size (int): Size of the source alphabet (including start, stop, and pad tokens).
        batch_first is set to be default True, more intuitive to reason about dimensionality if batching 
            dimension is first
        """

        super().__init__()

        ### MODIFY BELOW ###

        ### MODIFY ABOVE ###
        
        self.network = Transformer( src_embed_layer, tgt_embed_layer,
                                    src_pad_token, tgt_pad_token, 
                                    src_forward_function, tgt_forward_function,
                                    d_model, nhead, num_encoder_layers, num_decoder_layers,
                                    dim_feedforward, dropout, activation, custom_encoder,
                                    custom_decoder, target_size, source_size,
                                    layer_norm_eps, batch_first, norm_first, device, dtype)
        self.initialize_weights()
        self.freeze_components = freeze_components
        self.device = device
        self.dtype = dtype

    def initialize_weights(self) -> None:
        """Initializes network weights
        Non-1D parameters are initialized using Xavier initialization
        """
        for p in self.network.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def freeze(self) -> None:
        """Disables gradients for specific components of the network
        
        Args:
            components: A list of strings corresponding to the model components
                to freeze, e.g. src_embed, tgt_embed.
        """
        #TODO: This will need careful testing
        if self.freeze_components is not None:
            for component in self.freeze_components:
                if hasattr(self.network, component):
                    for param in getattr(self.network, component).parameters():
                        param.requires_grad = False
    
    def forward(self, 
                x: Tuple[Tensor, Tuple], 
                y: Tuple[Tensor, Tensor]) -> Tensor:
        return self.network(x, y)
    
    def get_loss(self,
                 x: Tuple[Tensor, Tuple], 
                 y: Tuple[Tensor], 
                 loss_fn: Callable[[Tensor, Tensor], Tensor]) -> Tensor:
        return self.network.get_loss(x, y, loss_fn)

In the following sections we will load the substructure to structure model you have been training and attempt to use it for structure prediction. The subsequent sections will only run if you have implemented the model correctly and so will serve as a check for your implementation.

## Loading our substructure to structure model

While you were implementing your model, hopefully your model has also been training. If it has trained to a point where the loss over the validation set has seemingly plateaud, copy the contents of the "checkpoints" subfolder into a folder in the directory here named "model1". If training has not seemingly saturated, you can instead use the pretrained model provided (see "transformer_checkpoint.ckpt" provided by the NMR2Struct package [here](https://github.com/MarklandGroup/NMR2Struct/tree/main/checkpoints)).

Below we will try to load the trained model using just the code in this notebook, which is a stripped down reimplementation of the relevant code from the NMR2Struct package that we used to train this model. If you receive an error here, you will likely need to debug your implementation of the model in the previous section.

In [ ]:
listdoc =  yaml.safe_load(open('model1/full_inference_config.yaml', 'r'))
model_args = listdoc['model']
model_config = model_args['model_args']

model = TransformerModel(dtype=torch.float32, device=torch.device('cpu'), **model_config)
ckpt = torch.load('model1/model_epoch=397_loss=0.07503651.pt', map_location=torch.device('cpu'))['model_state_dict']
model.load_state_dict(ckpt)
model.eval()

## Inference with the substructure to structure model

With the model loaded, let us now attempt to use it to predict molecular structure given the constituent substructures. For our model, instead of passing it a binary encoding of the substructures (as is specified by the substructures.h5 dataset) we instead pass a zero-padded array that lists in order of index which substructures are present. We 1-index these listed substructures because we reserve zero for padding. Complete the function below to apply this conversion to the substructure input data. If implemented correctly, running the code block below should output the following array for the example provided:

```
array([104, 167, 171, 180, 183, 184, 211, 212, 217, 225, 231, 278, 594,
       646, 647, 649, 651, 652, 653, 654, 656, 717, 720, 721, 749, 765,
       869, 870, 879, 880, 887, 908, 912, 924, 949, 955, 957,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0])
```

In [ ]:
def substruct_transform(substructures, max_len, pad_token=0):
    """Transforms the input binary substructure array into shifted and padded 1-indexed array"""
    ### MODIFY BELOW ###

    ### MODIFY ABOVE ###


i = 10000
hf = h5py.File('data/substructures_test.h5', 'r')
isubs = hf['substructure_labels'][i]
hf.close()

max_len = 74
substruct_transform(isubs, max_len)

Now let us run the code block below to use our Transformer model to make a prediction for the first character of the molecule's SMILES string given the list of substructures for that molecule. Which character does our model predict to most likely be the first character?

In [ ]:
alphabet = np.load('model1/alphabet.npy')
start_token = 22
stop_token = 23

working_x = torch.tensor(substruct_transform(isubs, max_len))[None,:]
working_y = torch.tensor([start_token])[None,:]
working_token_probs = torch.tensor([])

x = (working_x, None)
y = (working_y, None)
next_pos = model(x, y)
next_val = next_pos[:, -1, :]
char_probs = torch.nn.functional.softmax(next_val, dim = -1)
char_probs

Now let us run prediction with our model again but now with the SMILES prefix that we now have after predicting for the first character in the previous code block to obtain the second predicted character of the SMILES string. Note that the number 22 serves as our SMILES string start character.

In [ ]:
def get_top_k_sample_batched(k_val: int | float , 
                             character_probabilities: Tensor) -> tuple[Tensor, Tensor]:
    """
    Generates the next character using top-k sampling scheme.

    In top-k sampling, the probability mass is redistributed among the
    top-k next tokens, where k is a hyperparameter. Once redistributed, 
    the next token is sampled from the top-k tokens.
    """
    top_values, top_indices = torch.topk(character_probabilities, k_val, sorted = True)
    #Take the sum of the top probabilities and renormalize
    tot_probs = top_values / torch.sum(top_values, dim = -1).reshape(-1, 1)
    #Sample from the top k probabilities. This represents a multinomial distribution
    try:
        assert(torch.allclose(torch.sum(tot_probs, dim = -1), torch.tensor(1.0)))
    except:
        print("Probabilities did not pass allclose check!")
        print(f"Sum of probs is {torch.sum(tot_probs)}")
    selected_index = torch.multinomial(tot_probs, 1)
    #For gather to work, both tensors have to have the same number of dimensions:
    if len(top_indices.shape) != len(selected_index.shape):
        top_indices = top_indices.reshape(selected_index.shape[0], -1)
    output = torch.gather(top_indices, -1, selected_index)
    output_token_probs = torch.gather(tot_probs, -1, selected_index)
    return output, output_token_probs


sample_val = 5
selected_indices, token_probs = get_top_k_sample_batched(sample_val, char_probs)

working_y = torch.cat((working_y, selected_indices), dim = -1)
working_token_probs = torch.cat((working_token_probs, token_probs), dim = -1)

print(working_y)
print(alphabet[working_y[0,1:]])

x = (working_x, None)
y = (working_y, None)
next_pos = model(x, y)
next_val = next_pos[:, -1, :]
char_probs = torch.nn.functional.softmax(next_val, dim = -1)

selected_indices, token_probs = get_top_k_sample_batched(sample_val, char_probs)

working_y = torch.cat((working_y, selected_indices), dim = -1)
working_token_probs = torch.cat((working_token_probs, token_probs), dim = -1)

print(working_y)
print(alphabet[working_y[0,1:]])

Write a routine that now makes autoregressive character predictions until we reach a stop token, which in our case will be the number 23, and thus have a completed molecule predicted. Have the routine return both the SMILES string of the generated structure as well as an array of all the token probabilities. Is your predicted molecule the correct one?

In [ ]:
def generate_structure(model, substruct_list, start_token=22, stop_token=23, max_len=74, sample_val=5, max_steps=100):
    ### MODIFY BELOW ###

    ### MODIFY ABOVE ###

pred_tokens, pred_token_probs = generate_structure(model, isubs)
pred_smi = ''
for ichar in alphabet[pred_tokens[0,1:-1]]:
    pred_smi+=ichar
print('Target: ' + ismi)
print('Predicted: ' + pred_smi)
print('Predicted Score: ' + str(np.log(pred_token_probs[0,:-1].detach().numpy()).sum()))

You may or may not have predicted for the correct molecule in running the previous code block the first time. Given that the predictions made by this model are probabilistic, if you run that code block again you might sample a different final molecular structure. In practice we will want to make a set of predictions for a given input and then rank them by which our model believes is most likely the actual molecular structure. Write a routine that accomplishes this below, predicting 10 molecules and saving for each molecule its SMILES string and its log probability

Be sure to exclude molecules that are to chemically feasible (can you use rdKit for this? Hint: try visualizing all your generated structures using the subsequent code block) and duplicate structures.

In [ ]:
def generate_k_structures(model, substruct_list, num_pred_per_tgt):
    ### MODIFY BELOW ###

    ### MODIFY ABOVE ###


sampled_smis, sampled_smi_scores = generate_k_structures(model, isubs, 10)
sampled_smi_scores

Use the below code block to visualize the 10 predicted molecules as ordered by their log probabilities

In [ ]:
sampled_mols = [Chem.MolFromSmiles(x) for x in sampled_smis]
Chem.Draw.MolsToGridImage(sampled_mols,molsPerRow=5,subImgSize=(200,200), legends=[str(x) for x in sampled_smis])